In [1]:
from DatasetHandler.BiwiBrowser import *

/home/mcicek/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def scaleAnnoByScalers(labels, scalers):
    return ((labels - scalers[0]) - scalers[1]) / scalers[1]

In [3]:
def labelFramesForSubj(frames, annos, timesteps = None, overlapping = False, scaling = True, scalers = None):
    frames = {n: f for n, f in frames}
    keys = sorted(frames & annos.keys())
    inputMatrix = numpy.stack(itemgetter(*keys)(frames))
    labels = numpy.stack(itemgetter(*keys)(annos))
    if scaling: # scaleX()
        #inputMatrix, labels = inputMatrix, scaleY(labels)
        if scalers != None: labels = scaleAnnoByScalers(labels, scalers)
    if timesteps != None:
        inputMatrix, labels = reshaper(inputMatrix, labels, timesteps, overlapping)
    return inputMatrix, labels

In [4]:
def readBIWIDataset(dataFolder = BIWI_Data_folder, labelsTarFile = BIWI_Lebels_file, subjectList = None, timesteps = None, overlapping = False, scaling = True, preprocess_input = None):
    if subjectList == None: subjectList = [s for s in range(1, 25)]
    biwiFrames = readBIWI_Frames(dataFolder = dataFolder, subjectList = subjectList, preprocess_input = preprocess_input)
    biwiAnnos = readBIWI_Annos(tarFile = labelsTarFile, subjectList = subjectList)
    scalers = BIWI_Lebel_Scalers #getAnnoScalers(biwiAnnos, tarFile = labelsTarFile, subjectList = subjectList)
    labeledFrames = lambda frames, labels: labelFramesForSubj(frames, labels, timesteps, overlapping, scaling, scalers)
    biwi = (labeledFrames(frames, biwiAnnos[subj]) for subj, frames in biwiFrames.items())
    print('All frames and annotations from ' + str(len(subjectList)) + ' datasets have been read by ' + now())
    return biwi   

In [2]:
biwi = readBIWIDataset(subjectList = [1])

All frames and annotations from 1 datasets have been read by 2019-01-29 00:31:44.160664


In [11]:
biwiAnnos = readBIWI_Annos()
maxs = numpy.array([float('-inf') for i in range(6)])
mins = numpy.array([float('inf') for i in range(6)])
for subj, annos in biwiAnnos.items():
    #print(subj)
    for name, anno in annos.items():
        #anno = anno + (-1 * numpy.array([-92.04399871826172, -87.70659637451172, 754.1820068359375, -84.35343170166016, -66.95036315917969, -69.62425994873047]))
        #anno = anno - (numpy.array([323.39600372314453, 334.3906021118164, 543.2679443359375, 137.90052795410156, 143.84380340576172, 132.99221801757812]) / 2)
        #anno = anno / (numpy.array([323.39600372314453, 334.3906021118164, 543.2679443359375, 137.90052795410156, 143.84380340576172, 132.99221801757812]) / 2)
        for i in range(6):
            if anno[i] > maxs[i]:
                maxs[i] = anno[i]
            if anno[i] < mins[i]:
                mins[i] = anno[i]
print('Mins:', mins)
print('Maxs:', maxs)
#return [mins, maxs]

Mins: [-92.04399872 -87.70659637 754.18200684 -84.3534317  -66.95036316
 -69.62425995]
Maxs: [ 231.352005    246.68400574 1297.44995117   53.54709625   76.89344025
   63.36795807]


In [9]:
getMaxMinValuesOfAnnos()

[array([-92.04399872, -87.70659637, 754.18200684, -84.3534317 ,
        -66.95036316, -69.62425995]),
 array([ 231.352005  ,  246.68400574, 1297.44995117,   53.54709625,
          76.89344025,   63.36795807])]

In [8]:
BIWI_Lebel_Scalers

[array([-92.04399872, -87.70659637, 754.18200684, -84.3534317 ,
        -66.95036316, -69.62425995]),
 array([ 231.352005  ,  246.68400574, 1297.44995117,   53.54709625,
          76.89344025,   63.36795807])]

In [3]:
m, l = [], []
for subj, (inputMatrix, labels) in enumerate(biwi):
    m.append(inputMatrix)
    l.append(labels)
    print(subj+1, inputMatrix.shape, labels.shape)

1 (498, 224, 224, 3) (498, 6)


In [4]:
m, l = m[0], l[0]

In [5]:
l

array([[-0.11009043, -0.24473778, -0.53073992,  0.1103741 , -0.01913834,
         0.01042028],
       [-0.11065878, -0.24514926, -0.53016177,  0.11347951, -0.02214213,
         0.01082283],
       [-0.11118012, -0.24486456, -0.53044534,  0.11328992, -0.02025011,
         0.01128712],
       ...,
       [ 0.5317134 ,  0.07641781, -0.48777776,  0.83854968, -0.92924093,
        -0.15185888],
       [ 0.53393364,  0.07454336, -0.48819726,  0.84337212, -0.92867044,
        -0.15532004],
       [ 0.53770611,  0.09044452, -0.48696031,  0.82532875, -0.93205945,
        -0.14547584]])

In [29]:
def scaleAnnoByScalers(labels, scalers):
    return ((l - scalers[0]) - ((scalers[1]-scalers[0])/2)) / ((scalers[1]-scalers[0])/2)

In [30]:
labels = scaleAnnoByScalers(l, BIWI_Lebel_Scalers)

In [33]:
(labels * (scalers[1]-scalers[0])/2) + ((scalers[1]-scalers[0])/2) + scalers[0]

array([[ 51.8526001 ,  38.56969833, 881.64898682,  -7.7928443 ,
          3.59507298,  -2.43524265],
       [ 51.76070023,  38.50090027, 881.80603027,  -7.57872534,
          3.37903452,  -2.40847516],
       [ 51.67639923,  38.54850006, 881.72900391,  -7.59179783,
          3.51511192,  -2.37760139],
       ...,
       [155.6309967 ,  92.26540375, 893.31896973,  42.41505432,
        -61.86123657, -13.22617531],
       [155.99000549,  91.95200348, 893.20501709,  42.74756241,
        -61.82020569, -13.45632935],
       [156.6000061 ,  94.61060333, 893.54101562,  41.50346756,
        -62.06394958, -12.80172825]])

In [6]:
unscaleAnnoByScalers(l, BIWI_Lebel_Scalers)

array([[ 51.8526001 ,  38.56969833, 881.64898682,  -7.7928443 ,
          3.59507298,  -2.43524265],
       [ 51.76070023,  38.50090027, 881.80603027,  -7.57872534,
          3.37903452,  -2.40847516],
       [ 51.67639923,  38.54850006, 881.72900391,  -7.59179783,
          3.51511192,  -2.37760139],
       ...,
       [155.6309967 ,  92.26540375, 893.31896973,  42.41505432,
        -61.86123657, -13.22617531],
       [155.99000549,  91.95200348, 893.20501709,  42.74756241,
        -61.82020569, -13.45632935],
       [156.6000061 ,  94.61060333, 893.54101562,  41.50346756,
        -62.06394958, -12.80172825]])

In [31]:
print(labels.max(), labels.min())

0.9727859045179674 -1.0


In [17]:
scalers = BIWI_Lebel_Scalers

In [18]:
scalers

[array([-92.04399872, -87.70659637, 754.18200684, -84.3534317 ,
        -66.95036316, -69.62425995]),
 array([ 231.352005  ,  246.68400574, 1297.44995117,   53.54709625,
          76.89344025,   63.36795807])]

In [19]:
l

array([[ 51.8526001 ,  38.56969833, 881.64898682,  -7.7928443 ,
          3.59507298,  -2.43524265],
       [ 51.76070023,  38.50090027, 881.80603027,  -7.57872534,
          3.37903452,  -2.40847516],
       [ 51.67639923,  38.54850006, 881.72900391,  -7.59179783,
          3.51511192,  -2.37760139],
       ...,
       [155.6309967 ,  92.26540375, 893.31896973,  42.41505432,
        -61.86123657, -13.22617531],
       [155.99000549,  91.95200348, 893.20501709,  42.74756241,
        -61.82020569, -13.45632935],
       [156.6000061 ,  94.61060333, 893.54101562,  41.50346756,
        -62.06394958, -12.80172825]])

In [20]:
l - scalers[0]

array([[143.89659882, 126.27629471, 127.46697998,  76.56058741,
         70.54543614,  67.1890173 ],
       [143.80469894, 126.20749664, 127.62402344,  76.77470636,
         70.32939768,  67.21578479],
       [143.72039795, 126.25509644, 127.54699707,  76.76163387,
         70.46547508,  67.24665856],
       ...,
       [247.67499542, 179.97200012, 139.13696289, 126.76848602,
          5.08912659,  56.39808464],
       [248.03400421, 179.65859985, 139.02301025, 127.10099411,
          5.13015747,  56.1679306 ],
       [248.64400482, 182.31719971, 139.35900879, 125.85689926,
          4.88641357,  56.8225317 ]])

In [22]:
(scalers[1]-scalers[0])/2

array([161.69800186, 167.19530106, 271.63397217,  68.95026398,
        71.9219017 ,  66.49610901])

In [23]:
(l - scalers[0]) - ((scalers[1]-scalers[0])/2)

array([[ -17.80140305,  -40.91900635, -144.16699219,    7.61032343,
          -1.37646556,    0.69290829],
       [ -17.89330292,  -40.98780441, -144.00994873,    7.82444239,
          -1.59250402,    0.71967578],
       [ -17.97760391,  -40.94020462, -144.0869751 ,    7.8113699 ,
          -1.45642662,    0.75054955],
       ...,
       [  85.97699356,   12.77669907, -132.49700928,   57.81822205,
         -66.83277512,  -10.09802437],
       [  86.33600235,   12.4632988 , -132.61096191,   58.15073013,
         -66.79174423,  -10.32817841],
       [  86.94600296,   15.12189865, -132.27496338,   56.90663528,
         -67.03548813,   -9.67357731]])

In [ ]:
(l - scalers[0]) - ((scalers[1]-scalers[0])/2)[0]

In [25]:
((l - scalers[0]) - ((scalers[1]-scalers[0])/2)) / ((scalers[1]-scalers[0])/2)

array([[-0.11009043, -0.24473778, -0.53073992,  0.1103741 , -0.01913834,
         0.01042028],
       [-0.11065878, -0.24514926, -0.53016177,  0.11347951, -0.02214213,
         0.01082283],
       [-0.11118012, -0.24486456, -0.53044534,  0.11328992, -0.02025011,
         0.01128712],
       ...,
       [ 0.5317134 ,  0.07641781, -0.48777776,  0.83854968, -0.92924093,
        -0.15185888],
       [ 0.53393364,  0.07454336, -0.48819726,  0.84337212, -0.92867044,
        -0.15532004],
       [ 0.53770611,  0.09044452, -0.48696031,  0.82532875, -0.93205945,
        -0.14547584]])